# Validation Demo

This example will show how to evaluate Salient's probabilistic forecasts.


In [ ]:
import xarray as xr
import pandas as pd
import numpy as np

import sys
import os

try:
    import salientsdk as sk
except ModuleNotFoundError as e:
    if os.path.exists("../salientsdk"):
        sys.path.append(os.path.abspath(".."))
        import salientsdk as sk
    else:
        raise ModuleNotFoundError("Install salient SDK with: pip install salientsdk")

# Prevent wrapping on tables for readability
pd.set_option("display.width", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.expand_frame_repr", False)

# The variable that we'll be evaluating.
var = "temp"
fld = "anom"
timescale = "sub-seasonal"
start_date = "2021-01-01"
end_date = "2021-12-31"
ref_model = "clim"

sk.set_file_destination("validation_example")
sk.login("username", "password")

## Set geographic bounds

The Salient SDK uses a "Location" object to specify the geographic bounds of a request.


In [ ]:
if True:  # lat/lon
    loc = sk.Location(37.7749, -122.4194)
elif False:  # "shapefile"
    # Not currently supported by forecast_timeseries
    shp_file = sk.upload_bounding_box(-109.1, -102.0, 37.8, 41.0, "Colorado")
    loc = sk.Location(shapefile=shp_file)
elif True:  # "location_file"
    loc_file = sk.upload_location_file(
        lats=[37.7749, 33.9416, 32.7336],
        lons=[-122.4194, -118.4085, -117.1897],
        names=["SFO", "LAX", "SAN"],
        geoname="CA_Airports",
    )
    loc = sk.Location(location_file=loc_file)
else:
    raise ValueError("Invalid location type")

print(loc)

(37.7749, -122.4194)


## Forecast

The [`forecast_timeseries`](https://sdk.salientpredictions.com/api/#salientsdk.forecast_timeseries) API endpoint and SDK function returns Salient's native temporally granular weekly/monthly/quarterly forecasts.


In [ ]:
date_range = pd.date_range(start=start_date, end=end_date, freq="W").strftime("%Y-%m-%d").tolist()

fcst = sk.forecast_timeseries(
    loc=loc,
    variable=var,
    field=fld,
    date=date_range,
    timescale=timescale,
    model=["blend", ref_model],
    reference_clim="30_yr",  # this is the climatology used by data_timeseries
    verbose=False,
    force=False,
)

print(fcst)

                                             file_name        date  model
0    validation_example/forecast_timeseries_6e149e8...  2021-01-03  blend
1    validation_example/forecast_timeseries_b5b2e5d...  2021-01-03   clim
2    validation_example/forecast_timeseries_7f3f3be...  2021-01-10  blend
3    validation_example/forecast_timeseries_0390b0f...  2021-01-10   clim
4    validation_example/forecast_timeseries_9568136...  2021-01-17  blend
..                                                 ...         ...    ...
99   validation_example/forecast_timeseries_244dd5a...  2021-12-12   clim
100  validation_example/forecast_timeseries_5e5a25e...  2021-12-19  blend
101  validation_example/forecast_timeseries_34c4052...  2021-12-19   clim
102  validation_example/forecast_timeseries_334bb13...  2021-12-26  blend
103  validation_example/forecast_timeseries_121fb74...  2021-12-26   clim

[104 rows x 3 columns]


## Historical

Download daily historical values from [`data_timeseries`](https://sdk.salientpredictions.com/api/#salientsdk.data_timeseries) and then aggregate to match the forecasts, so that we can ensure that all forecasts use the same dates.


In [ ]:
duration = {"sub-seasonal": 7 * 5, "seasonal": 30 * 3, "long-range": 90 * 4}[timescale]
hist = sk.data_timeseries(
    loc=loc,
    variable=var,
    field=fld,
    start=start_date,
    end=np.datetime64(end_date) + np.timedelta64(duration, "D"),
    frequency="daily",
    # reference_clim="30_yr",  implicitly uses 30 yr climatology
    verbose=False,
    force=False,
)
print(xr.load_dataset(hist))

<xarray.Dataset> Size: 6kB
Dimensions:  (time: 400, location: 1)
Coordinates:
  * time     (time) datetime64[ns] 3kB 2021-01-01 2021-01-02 ... 2022-02-04
    lat      (location) float64 8B 37.77
    lon      (location) float64 8B -122.4
Dimensions without coordinates: location
Data variables:
    anom     (time, location) float64 3kB -0.2312 1.134 ... -0.3179 -0.7596
Attributes:
    long_name:   2 metre temperature
    units:       degC
    clim_start:  1990-01-01
    clim_end:    2019-12-31


## Calculate Skill Metrics

Compare the forecast and observed datasets to see how well they match.


In [ ]:
skill_fcst = sk.skill.crps(observations=hist, forecasts=fcst[fcst["model"] == "blend"])
print(skill_fcst)

<xarray.DataArray 'crps' (lead_weekly: 5, location: 1)> Size: 40B
array([[0.51113442],
       [0.48719398],
       [0.56312406],
       [0.56776205],
       [0.5550928 ]])
Coordinates:
    lat          (location) float64 8B 37.77
    lon          (location) float64 8B -122.4
  * lead_weekly  (lead_weekly) int32 20B 1 2 3 4 5
Dimensions without coordinates: location
Attributes:
    long_name:  Average Temperature Anomaly CRPS
    units:      degC


### Calculate Relative Skill

CRPS shows skill without context. A "skill score" will compare two different skills for a relative value. In the example below, we will compare the Salient blend with climatology (historical averages).


In [ ]:
skill_ref = sk.skill.crps(observations=hist, forecasts=fcst[fcst["model"] == ref_model])

skill_score = sk.skill.crpss(forecast=skill_fcst, reference=skill_ref)

print(skill_score)

<xarray.DataArray 'crpss' (lead_weekly: 5, location: 1)> Size: 40B
array([[ 0.14086106],
       [ 0.183023  ],
       [ 0.05724795],
       [-0.01447167],
       [-0.00077589]])
Coordinates:
    lat          (location) float64 8B 37.77
    lon          (location) float64 8B -122.4
  * lead_weekly  (lead_weekly) int32 20B 1 2 3 4 5
Dimensions without coordinates: location
Attributes:
    long_name:  Average Temperature Anomaly CRPS Skill


### Compare to pre-computed skill


In [ ]:
skill_summ = sk.hindcast_summary(
    loc=loc,
    metric="crps",
    variable=var,
    timescale=timescale,
    reference=ref_model,
    split_set="test",
)

print(pd.read_csv(skill_summ))

     Lead      Reference Model  Reference CRPS  Salient CRPS  Salient CRPS Skill Score (%)
0  Week 1  30 year Climatology             0.9          0.46                          49.3
1  Week 2  30 year Climatology             0.9          0.65                          27.9
2  Week 3  30 year Climatology             0.9          0.78                          13.5
3  Week 4  30 year Climatology             0.9          0.81                          10.1
4  Week 5  30 year Climatology             0.9          0.82                           8.8
